In [39]:
# Load the extension that allows us to compile CUDA code in python notebooks
# Documentation is here: https://nvcc4jupyter.readthedocs.io/en/latest/
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter



  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-9cgjep6z
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-9cgjep6z
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


# 新段落

In [40]:
import torchvision
import os

def download_mnist_dataset():
    # 创建目录
    os.makedirs("train_mnist/MNIST/raw", exist_ok=True)
    os.makedirs("test_mnist/MNIST/raw", exist_ok=True)

    # 下载训练数据
    train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True)
    test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True)

    print("MNIST dataset downloaded successfully.")

# 调用函数下载数据集
download_mnist_dataset()

MNIST dataset downloaded successfully.


In [41]:
'''DO NOT UNCOMMENT THIS CELL unless you are running this notebook on Google Colab'''
# from google.colab import drive
# drive.mount('/content/drive/', force_remount=True)


'DO NOT UNCOMMENT THIS CELL unless you are running this notebook on Google Colab'

In [42]:
%%cuda_group_save -g "knn" -n "main.cu"

// Required header files / 所需的头文件
#include <iostream>     // For input/output operations / 用于输入输出操作
#include <fstream>      // For file operations / 用于文件操作
#include <vector>       // For vector container / 用于向量容器
#include <string>       // For string operations / 用于字符串操作
#include <cstring>      // For C-style string operations / 用于C风格字符串操作
#include <algorithm>    // For algorithms like max_element / 用于算法如max_element
#include <cuda_runtime.h> // For CUDA operations / 用于CUDA操作
#include <cfloat>
#include <chrono>    // For timing execution / 用于执行时间计算

// Constants definition / 常量定义
#define THREADS 256        // Number of threads per block / 每个块的线程数
#define IMAGESIZE 784      // Image size (28x28 = 784 pixels) / 图像大小 (28x28 = 784像素)

// Function to handle big-endian to little-endian conversion
// 处理大端序转小端序的函数
uint32_t swap32(uint32_t val) {
    val = ((val << 8) & 0xFF00FF00) | ((val >> 8) & 0xFF00FF);
    return (val << 16) | (val >> 16);
}

// structure to store training/testing samples
// 存储训练/测试样本的结构体
struct TrainingSample {
    int label;                  // The digit (0-9) / 数字标签 (0-9)
    float image[IMAGESIZE];     // Normalized pixel values / 归一化的像素值
};

struct KernelTiming {
    float data_transfer;
    float distance_calc;
    float sorting;
    int num_samples;
} timing = {0.0f, 0.0f, 0.0f, 0};

__global__ void bitonicSortStep(float* d_distances, int* d_labels, int j, int k, int num_samples) {
    unsigned int i = threadIdx.x + blockDim.x * blockIdx.x;
    if (i >= num_samples) return;

    unsigned int ixj = i ^ j;

    if (ixj > i && ixj < num_samples) {
        // determine the sorting direction
        if ((i & k) == 0) {
            // sort in ascending order
            if (d_distances[i] > d_distances[ixj]) {
                // swap distances
                float temp_dist = d_distances[i];
                d_distances[i] = d_distances[ixj];
                d_distances[ixj] = temp_dist;

                // swap corresponding labels
                int temp_label = d_labels[i];
                d_labels[i] = d_labels[ixj];
                d_labels[ixj] = temp_label;
            }
        } else {
            // sort in descending order
            if (d_distances[i] < d_distances[ixj]) {
                // swap distances
                float temp_dist = d_distances[i];
                d_distances[i] = d_distances[ixj];
                d_distances[ixj] = temp_dist;

                // swap corresponding labels
                int temp_label = d_labels[i];
                d_labels[i] = d_labels[ixj];
                d_labels[ixj] = temp_label;
            }
        }
    }
}

void bitonicSort(float* d_distances, int* d_labels, int num_samples, cudaStream_t stream) {
    // Calculate the next power of two
    int pow2_size = 1;
    while (pow2_size < num_samples) pow2_size <<= 1;

    // Pad the distances and labels with maximum values
    int padded_size = pow2_size;
    if (padded_size > num_samples) {
        float max_distance = FLT_MAX;
        int max_label = -1; // Use an invalid label for padding

        // Create temporary arrays for padding
        float* h_pad_distances;
        int* h_pad_labels;
        cudaMallocHost(&h_pad_distances, (padded_size - num_samples) * sizeof(float));
        cudaMallocHost(&h_pad_labels, (padded_size - num_samples) * sizeof(int));
        
        for (int i = 0; i < padded_size - num_samples; ++i) {
            h_pad_distances[i] = max_distance;
            h_pad_labels[i] = max_label;
        }

        // Copy padding data to device asynchronously
        cudaMemcpyAsync(d_distances + num_samples, 
                       h_pad_distances, 
                       (padded_size - num_samples) * sizeof(float), 
                       cudaMemcpyHostToDevice,
                       stream);
        cudaMemcpyAsync(d_labels + num_samples, 
                       h_pad_labels, 
                       (padded_size - num_samples) * sizeof(int), 
                       cudaMemcpyHostToDevice,
                       stream);

        // Free temporary host arrays
        cudaFreeHost(h_pad_distances);
        cudaFreeHost(h_pad_labels);
    }

    // Set up grid and block dimensions
    dim3 block(THREADS);
    dim3 grid((padded_size + block.x - 1) / block.x);

    // Main sorting loops
    for (int k = 2; k <= pow2_size; k <<= 1) {
        for (int j = k >> 1; j > 0; j >>= 1) {
            bitonicSortStep<<<grid, block, 0, stream>>>(
                d_distances, d_labels, j, k, padded_size
            );
        }
    }
}


// 1. 优化的向量化距离计算核函数
__global__ void computeEuclideanDistances(float* d_images, float* d_testImage,
                                        float* d_distances, int* d_labels,
                                        int* d_train_labels, int num_samples) {
    extern __shared__ float shared_mem[];
    float* shared_test = shared_mem;
    
    int tid = threadIdx.x;
    int bid = blockIdx.x;
    int idx = bid * blockDim.x + tid;
    
    // 使用协作加载来提高内存访问效率
    for (int i = tid; i < IMAGESIZE; i += blockDim.x) {
        shared_test[i] = d_testImage[i];
    }
    __syncthreads();
    
    if (idx < num_samples) {
        float sum = 0.0f;
        
        // 使用循环展开和向量加载来优化计算
        float4* img_vec = (float4*)(&d_images[idx * IMAGESIZE]);
        float4* test_vec = (float4*)shared_test;
        
        #pragma unroll 16
        for (int i = 0; i < IMAGESIZE/4; i++) {
            float4 diff;
            float4 img = img_vec[i];
            float4 test = test_vec[i];
            
            diff.x = img.x - test.x;
            diff.y = img.y - test.y;
            diff.z = img.z - test.z;
            diff.w = img.w - test.w;
            
            sum += diff.x * diff.x + diff.y * diff.y + 
                   diff.z * diff.z + diff.w * diff.w;
        }
        
        // Handle remaining elements
        for (int i = (IMAGESIZE/4)*4; i < IMAGESIZE; i++) {
            float diff = d_images[idx * IMAGESIZE + i] - shared_test[i];
            sum += diff * diff;
        }
        
        d_distances[idx] = sqrtf(sum);
        d_labels[idx] = d_train_labels[idx];
    }
}

// function to load MNIST dataset in IDX format
// 加载IDX格式MNIST数据集的函数
bool loadMNISTImages(const std::string& image_path, const std::string& label_path,
                    std::vector<TrainingSample>& samples) {
    // Open image file / 打开图像文件
    std::ifstream image_file(image_path, std::ios::binary);
    if (!image_file) {
        std::cerr << "Cannot open image file: " << image_path << std::endl;
        return false;
    }

    // Open label file / 打开标签文件
    std::ifstream label_file(label_path, std::ios::binary);
    if (!label_file) {
        std::cerr << "Cannot open label file: " << label_path << std::endl;
        return false;
    }

    // Read image file header / 读取图像文件头
    uint32_t magic, num_items, num_rows, num_cols;
    image_file.read(reinterpret_cast<char*>(&magic), sizeof(magic));
    image_file.read(reinterpret_cast<char*>(&num_items), sizeof(num_items));
    image_file.read(reinterpret_cast<char*>(&num_rows), sizeof(num_rows));
    image_file.read(reinterpret_cast<char*>(&num_cols), sizeof(num_cols));

    // Convert from big-endian to host endian / 从大端序转换为主机字节序
    magic = swap32(magic);
    num_items = swap32(num_items);
    num_rows = swap32(num_rows);
    num_cols = swap32(num_cols);

    // Verify image file format / 验证图像文件格式
    if (magic != 0x803) {
        std::cerr << "Invalid image file format" << std::endl;
        return false;
    }

    // Read label file header / 读取标签文件头
    uint32_t label_magic, num_labels;
    label_file.read(reinterpret_cast<char*>(&label_magic), sizeof(label_magic));
    label_file.read(reinterpret_cast<char*>(&num_labels), sizeof(num_labels));

    // Convert label file header / 转换标签文件头
    label_magic = swap32(label_magic);
    num_labels = swap32(num_labels);

    // Verify label file format / 验证标签文件格式
    if (label_magic != 0x801) {
        std::cerr << "Invalid label file format" << std::endl;
        return false;
    }

    // Check consistency between images and labels / 检查图像和标签数量是否一致
    if (num_items != num_labels) {
        std::cerr << "Number of images doesn't match number of labels" << std::endl;
        return false;
    }

    // Prepare storage / 准备存储空间
    samples.resize(num_items);
    std::vector<unsigned char> pixels(num_rows * num_cols);

    // Read and process each sample / 读取并处理每个样本
    for (uint32_t i = 0; i < num_items; ++i) {
        // Read label / 读取标签
        unsigned char label;
        label_file.read(reinterpret_cast<char*>(&label), 1);
        samples[i].label = static_cast<int>(label);

        // Read image / 读取图像
        image_file.read(reinterpret_cast<char*>(pixels.data()), pixels.size());

        // Normalize pixel values to [0,1] / 将像素值归一化到[0,1]范围
        for (size_t j = 0; j < pixels.size(); ++j) {
            samples[i].image[j] = static_cast<float>(pixels[j]) / 255.0f;
        }

        // Show progress / 显示进度
        if (i % 1000 == 0) {
            std::cout << "\rLoading data: " << (i * 100.0f / num_items) << "%" << std::flush;
        }
    }
    std::cout << "\rLoading data: 100%" << std::endl;

    return true;
}

int main() {
    // Timing structure for kernel breakdown
    struct KernelTiming {
        float data_transfer;
        float distance_calc;
        float sorting;
        int num_samples;
    } timing = {0.0f, 0.0f, 0.0f, 0};

    // Start timing
    auto start_time = std::chrono::high_resolution_clock::now();
    
    // Load data
    std::vector<TrainingSample> train_samples;
    std::vector<TrainingSample> test_samples;

    if (!loadMNISTImages("./data/MNIST/raw/train-images-idx3-ubyte",
                        "./data/MNIST/raw/train-labels-idx1-ubyte",
                        train_samples)) {
        return -1;
    }
    std::cout << "Successfully loaded " << train_samples.size() << " training samples." << std::endl;

    if (!loadMNISTImages("./data/MNIST/raw/t10k-images-idx3-ubyte",
                        "./data/MNIST/raw/t10k-labels-idx1-ubyte",
                        test_samples)) {
        return -1;
    }
    std::cout << "Successfully loaded " << test_samples.size() << " testing samples." << std::endl;

    int num_trainsamples = train_samples.size();
    int num_testsamples = test_samples.size();

    // Allocate page-locked memory for better transfer speed
    float* h_train_images;
    int* h_train_labels;
    cudaMallocHost(&h_train_images, num_trainsamples * IMAGESIZE * sizeof(float));
    cudaMallocHost(&h_train_labels, num_trainsamples * sizeof(int));

    // Copy data to page-locked memory
    for (int i = 0; i < num_trainsamples; ++i) {
        h_train_labels[i] = train_samples[i].label;
        std::memcpy(&h_train_images[i * IMAGESIZE], train_samples[i].image, sizeof(float) * IMAGESIZE);
    }

    // Allocate GPU memory
    float* d_train_images;
    int* d_train_labels;
    cudaMalloc(&d_train_images, num_trainsamples * IMAGESIZE * sizeof(float));
    cudaMalloc(&d_train_labels, num_trainsamples * sizeof(int));

    // Copy training data to GPU
    cudaMemcpy(d_train_images, h_train_images, num_trainsamples * IMAGESIZE * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_train_labels, h_train_labels, num_trainsamples * sizeof(int), cudaMemcpyHostToDevice);

    // KNN parameters
    const int k = 10;
    int correct_predictions = 0;

    // Create CUDA streams for parallel processing
    const int NUM_STREAMS = 4;
    cudaStream_t streams[NUM_STREAMS];
    for (int i = 0; i < NUM_STREAMS; i++) {
        cudaStreamCreate(&streams[i]);
    }

    // Allocate memory for each stream
    float* d_test_images[NUM_STREAMS];
    float* d_distances[NUM_STREAMS];
    int* d_sort_labels[NUM_STREAMS];
    float* h_distances[NUM_STREAMS];
    int* h_labels[NUM_STREAMS];
    
    for (int i = 0; i < NUM_STREAMS; i++) {
        cudaMalloc(&d_test_images[i], IMAGESIZE * sizeof(float));
        cudaMalloc(&d_distances[i], num_trainsamples * sizeof(float));
        cudaMalloc(&d_sort_labels[i], num_trainsamples * sizeof(int));
        cudaMallocHost(&h_distances[i], k * sizeof(float));
        cudaMallocHost(&h_labels[i], k * sizeof(int));
    }

    // Configure kernel parameters
    int threadsPerBlock = 256;
    int blocksPerGrid = (num_trainsamples + threadsPerBlock - 1) / threadsPerBlock;
    size_t shared_mem_size = IMAGESIZE * sizeof(float);

    // Process test samples in batches using streams
    for (int t = 0; t < num_testsamples; t += NUM_STREAMS) {
        // Launch work on each stream
        for (int s = 0; s < NUM_STREAMS && (t + s) < num_testsamples; s++) {
            int current_sample = t + s;
            
            // Create timing events
            cudaEvent_t start_transfer, stop_transfer;
            cudaEvent_t start_distance, stop_distance;
            cudaEvent_t start_sort, stop_sort;
            
            cudaEventCreate(&start_transfer);
            cudaEventCreate(&stop_transfer);
            cudaEventCreate(&start_distance);
            cudaEventCreate(&stop_distance);
            cudaEventCreate(&start_sort);
            cudaEventCreate(&stop_sort);

            // Time data transfer
            cudaEventRecord(start_transfer, streams[s]);
            cudaMemcpyAsync(d_test_images[s], 
                          test_samples[current_sample].image,
                          IMAGESIZE * sizeof(float), 
                          cudaMemcpyHostToDevice,
                          streams[s]);
            cudaEventRecord(stop_transfer, streams[s]);

            // Time distance calculation
            cudaEventRecord(start_distance, streams[s]);
            computeEuclideanDistances<<<blocksPerGrid, threadsPerBlock, shared_mem_size, streams[s]>>>(
                d_train_images,
                d_test_images[s],
                d_distances[s],
                d_sort_labels[s],
                d_train_labels,
                num_trainsamples
            );
            cudaEventRecord(stop_distance, streams[s]);

            // Time sorting
            cudaEventRecord(start_sort, streams[s]);
            bitonicSort(d_distances[s], d_sort_labels[s], num_trainsamples, streams[s]);
            cudaEventRecord(stop_sort, streams[s]);

            cudaMemcpyAsync(h_distances[s], d_distances[s],
                          k * sizeof(float), cudaMemcpyDeviceToHost,
                          streams[s]);
            cudaMemcpyAsync(h_labels[s], d_sort_labels[s],
                          k * sizeof(int), cudaMemcpyDeviceToHost,
                          streams[s]);

            // Calculate timing for this iteration
            float transfer_time, distance_time, sort_time;
            cudaEventSynchronize(stop_transfer);
            cudaEventSynchronize(stop_distance);
            cudaEventSynchronize(stop_sort);
            
            cudaEventElapsedTime(&transfer_time, start_transfer, stop_transfer);
            cudaEventElapsedTime(&distance_time, start_distance, stop_distance);
            cudaEventElapsedTime(&sort_time, start_sort, stop_sort);

            // Accumulate times
            timing.data_transfer += transfer_time;
            timing.distance_calc += distance_time;
            timing.sorting += sort_time;
            timing.num_samples++;

            // Cleanup timing events
            cudaEventDestroy(start_transfer);
            cudaEventDestroy(stop_transfer);
            cudaEventDestroy(start_distance);
            cudaEventDestroy(stop_distance);
            cudaEventDestroy(start_sort);
            cudaEventDestroy(stop_sort);
        }

        // Process results for this batch
        for (int s = 0; s < NUM_STREAMS && (t + s) < num_testsamples; s++) {
            cudaStreamSynchronize(streams[s]);
            
            int current_sample = t + s;
            int test_label = test_samples[current_sample].label;

            std::vector<int> labelCount(10, 0);
            for (int i = 0; i < k; ++i) {
                if (h_labels[s][i] >= 0 && h_labels[s][i] < 10) {
                    labelCount[h_labels[s][i]]++;
                }
            }

            int predictedLabel = std::distance(labelCount.begin(),
                                            std::max_element(labelCount.begin(), labelCount.end()));

            if (predictedLabel == test_label) {
                correct_predictions++;
            }

            if (current_sample % 1000 == 0) {
                float current_accuracy = (float)correct_predictions / (current_sample + 1) * 100.0f;
                std::cout << "\rProcessing: " << current_sample << "/" << num_testsamples
                         << " (Accuracy: " << current_accuracy << "%)" << std::flush;
            }
        }
    }

    // Calculate final results
    float accuracy = (float)correct_predictions / num_testsamples * 100.0f;
    auto end_time = std::chrono::high_resolution_clock::now();
    auto duration = std::chrono::duration_cast<std::chrono::milliseconds>(end_time - start_time);

    // Print all results
    std::cout << "\n\nFinal Results:" << std::endl;
    std::cout << "Total test samples: " << num_testsamples << std::endl;
    std::cout << "Correct predictions: " << correct_predictions << std::endl;
    std::cout << "Accuracy: " << accuracy << "%" << std::endl;

    std::cout << "\nKernel Timing Breakdown:" << std::endl;
    std::cout << "Average Data Transfer Time: " << timing.data_transfer / timing.num_samples << " ms" << std::endl;
    std::cout << "Average Distance Calculation Time: " << timing.distance_calc / timing.num_samples << " ms" << std::endl;
    std::cout << "Average Sorting Time: " << timing.sorting / timing.num_samples << " ms" << std::endl;
    std::cout << "Total Data Transfer Time: " << timing.data_transfer << " ms" << std::endl;
    std::cout << "Total Distance Calculation Time: " << timing.distance_calc << " ms" << std::endl;
    std::cout << "Total Sorting Time: " << timing.sorting << " ms" << std::endl;
    
    std::cout << "\nTotal execution time: " << duration.count() / 1000.0 << " seconds" << std::endl;

    // Cleanup
    for (int i = 0; i < NUM_STREAMS; i++) {
        cudaFree(d_test_images[i]);
        cudaFree(d_distances[i]);
        cudaFree(d_sort_labels[i]);
        cudaFreeHost(h_distances[i]);
        cudaFreeHost(h_labels[i]);
        cudaStreamDestroy(streams[i]);
    }

    cudaFreeHost(h_train_images);
    cudaFreeHost(h_train_labels);
    cudaFree(d_train_images);
    cudaFree(d_train_labels);

    return 0;
}

In [43]:
%cuda_group_run --group "knn" --compiler-args "-O3 -g -std=c++20 -arch=sm_75"

Loading data: 100%
Loading data: 100%
Training samples: 10000
Test samples: 1000
Processed 0 test samples
Processed 100 test samples
Processed 200 test samples
Processed 300 test samples
Processed 400 test samples
Processed 500 test samples
Processed 600 test samples
Processed 700 test samples
Processed 800 test samples
Processed 900 test samples
Total test samples: 1000
Correct predictions: 477
Accuracy: 47.70%
Total execution time: 0.8644 seconds
Prediction time: 0.6952 seconds

